In [1]:
import cv2
import numpy as np
from scipy.signal import convolve2d
from tqdm import tqdm

In [2]:
#Configuration

K = 5 #Number of mixture models
T = 0.2 #Min portion of data accounted for Background
alpha=0.5 #pi update param
eps = 1e-5
low_prior_weight = 0.01
inf = 1e9
high_variance=1
in_video_file_path = 'Data/umcp.mpg' #path to input audio
out_path = '' #directory for foreground and background videos
frame_size = (240,352)

In [3]:
#model params

#For each pixel, K set of mean,var and pi are required; 
pi = np.ones((*frame_size,K))/K # pi-> (MxNxK)
mean = np.zeros((*frame_size,K))
var = np.ones((*frame_size,K))/K/K #Covariance matrix

params = {
    'pi':pi,
    'mean':mean,
    'var':var,
    'sorted_gaussians_idx': None,
    'B' : None
}

In [7]:
# utils

def KNN(x):
    win_size = 9
    win = np.ones((win_size,win_size))/(win_size*win_size)
    out = convolve2d(x.squeeze(),win,mode='same')
    out = (out>0.5)*1
    return out[:,:,np.newaxis]

def gpdf_gray(x,params):
    #gaussian probability density function
    mean = params['mean']
    var = params['var']
    
    mahalD = (x-mean)*(x-mean)/(var+eps)
    return 1/(np.sqrt(2*np.pi*(var+eps))) * np.exp(-mahalD/2)

def update_params(x,params,M,alpha=alpha):
    #M is a bool matrix(true and false)
    rho = alpha*gpdf_gray(x,params)
    for r in range(M.shape[0]):
        for c in range(M.shape[1]):
            no_match = True
            for k in range(M.shape[2]):
                if M[r][c][k]:
                    no_match = False
                    break
            if no_match:
                # replace the least probable distribution
                min_dist_idx = np.argmin(params['pi'][r][c])
                params['pi'][r][c][min_dist_idx] = low_prior_weight
                params['mean'][r][c][min_dist_idx] = x[r][c]
                params['var'][r][c][min_dist_idx] = high_variance
            else:
                for k in range(M.shape[2]):
                    params['pi'][r][c][k] = (1-alpha)*params['pi'][r][c][k] + alpha*M[r][c][k]
                    
                    if M[r][c][k] == True:
                        params['mean'][r][c][k] = (1-rho[r][c][k])*params['mean'][r][c][k] + \
                                                  rho[r][c][k]*x[r][c]

                        params['var'][r][c][k] = ((1-rho[r][c][k])*params['var'][r][c][k] \
                                              + rho[r][c][k]*((x[r][c]-params['mean'][r][c][k])**2))
                
                
            if params['var'][r][c][k] < 0:
                params['var'][r][c][k] *= (-1)
            # renormalise
    params['pi'] = params['pi']/np.sum(params['pi'],axis=2)[:,:,np.newaxis]
                    
            

def matching(x,params):
    #if x-mean < 2.5*var then its a match
    mean = params['mean']
    var = params['var']
    return np.abs(x-mean) < 2.5*var

def background_model(x,params):
    pi_by_var = -1*params['pi']/np.sqrt(params['var'] + eps)
    params['sorted_gaussians_idx'] = np.argsort(pi_by_var,axis=2)
    pi_sorted = -1*np.sort(-1*params['pi'],axis=2)
    M = frame_size[0]
    N = frame_size[1]
    params['B'] = np.zeros_like(x)
    for r in range(M):
        for c in range(N):
            cumsum = 0
            for k in range(K):
                cumsum += pi_sorted[r][c][k]
                params['B'][r][c] = k+1
                if cumsum > T:
                    break
                
            

    matched = matching(x,params)
    background = np.zeros_like(x)
    for r in range(M):
        for c in range(N):
            for k in range(params['B'][r][c][0]):
                if matched[r][c][params['sorted_gaussians_idx'][r][c][k]]:
                    # classify as background
                    background[r][c] = 1
                    break
    
    background = KNN(background)
    return background
                    

In [5]:
#MAIN

def main(outFolder):
    os.mkdir(f'{outFolder}/background')
    os.mkdir(f'{outFolder}/foreground')
    in_video = cv2.VideoCapture(in_video_file_path)
    idx=0
    with tqdm(total=1000) as pbar:
        while (in_video.isOpened()):
            rval, frame = in_video.read()
            idx +=1 
            if rval:
                #process frame
                frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)[:,:,2] #Take out I from HSI channel
                frame_gray = frame_gray[:,:,np.newaxis] #Append new axis for shape matching
                M = matching(frame_gray,params) #See if pixels match the model
                update_params(frame_gray,params,M) #update the parameters of the model based on matched value
                fg = 1 - background_model(frame_gray,params) #Get foreground mask by subtracting the background mask 

                #Back conversion into of I values to HSI
                frame_fg_gray = (fg.astype('uint8')*frame_gray).squeeze()
                frame_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
                frame_fg_hsv =np.concatenate((frame_hsv[:,:,:2],frame_fg_gray[:,:,np.newaxis]),axis=2)

                #Back Conversion to color
                frame_fg_color = cv2.cvtColor(frame_fg_hsv,cv2.COLOR_HSV2BGR)
                frame_bg_color = frame - frame_fg_color

                #writing images
                cv2.imwrite(f'{outFolder}/background/background_{idx:03d}.png',frame_bg_color)
                cv2.imwrite(f'{outFolder}/foreground/foreground_{idx:03d}.png',frame_fg_color)

            else:
                break
            pbar.update(1)
    !ffmpeg -framerate 30 -i $outFolder/background/background_%03d.png -c:v libx264 -profile:v high -crf 20 -pix_fmt yuv420p $outFolder/background.mp4
    !ffmpeg -framerate 30 -i $outFolder/foreground/foreground_%03d.png -c:v libx264 -profile:v high -crf 20 -pix_fmt yuv420p $outFolder/foreground.mp4
            
    shutil.rmtree(f'{outFolder}/background')
    shutil.rmtree(f'{outFolder}/foreground')
    in_video.release()

In [6]:
import os
import shutil
K = 5
T = 0.8
alpha = 0.1
os.mkdir(f'test_{K}_{T}_{alpha}')
main(f'test_{K}_{T}_{alpha}')

  0%|                                                                                                        | 1/1000 [00:02<35:33,  2.14s/it]/tmp/ipykernel_55814/3645168339.py:60: RuntimeWarning: invalid value encountered in sqrt
  pi_by_var = -1*params['pi']/np.sqrt(params['var'] + eps)
  0%|▏                                                                                                       | 2/1000 [00:05<47:25,  2.85s/it]/tmp/ipykernel_55814/3645168339.py:16: RuntimeWarning: invalid value encountered in sqrt
  return 1/(np.sqrt(2*np.pi*(var+eps))) * np.exp(-mahalD/2)
  0%|▎                                                                                                       | 3/1000 [00:08<47:38,  2.87s/it]/tmp/ipykernel_55814/3645168339.py:16: RuntimeWarning: overflow encountered in exp
  return 1/(np.sqrt(2*np.pi*(var+eps))) * np.exp(-mahalD/2)
100%|███████████████████████████████████████████████████████████████████████████████████████████████████▉| 999/1000 [1:08:05<00:04,  4

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

frame=  999 fps=156 q=-1.0 Lsize=    2411kB time=00:00:33.20 bitrate= 594.9kbits/s speed=5.18x    
video:2399kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.522254%
[libx264 @ 0x561f9cfe6700] frame I:5     Avg QP:18.69  size: 18711
[libx264 @ 0x561f9cfe6700] frame P:252   Avg QP:21.06  size:  4483
[libx264 @ 0x561f9cfe6700] frame B:742   Avg QP:25.85  size:  1661
[libx264 @ 0x561f9cfe6700] consecutive B-frames:  0.9%  0.2%  0.0% 98.9%
[libx264 @ 0x561f9cfe6700] mb I  I16..4: 11.1% 53.1% 35.8%
[libx264 @ 0x561f9cfe6700] mb P  I16..4:  1.1%  0.9%  2.7%  P16..4: 48.1% 15.7% 20.6%  0.0%  0.0%    skip:10.8%
[libx264 @ 0x561f9cfe6700] mb B  I16..4:  0.0%  0.1%  0.9%  B16..8: 33.5%  4.7%  3.8%  direct: 7.0%  skip:49.9%  L0:46.5% L1:46.8% BI: 6.7%
[libx264 @ 0x561f9cfe6700] 8x8 transform intra:22.6% inter:41.3%
[libx264 @ 0x561f9cfe6700] coded y,uvDC,uvAC intra: 75.0% 63.4% 41.4% inter: 16.7% 33.7% 8.9%
[libx264 @ 0x561f9cfe6700] i16 v,h,dc,p: 91%  2%  4%  3%


In [8]:
K = 5
T = 0.5
alpha = 0.01
os.mkdir(f'test_{K}_{T}_{alpha}')
main(f'test_{K}_{T}_{alpha}')

  0%|                                                                                                                | 0/1000 [00:00<?, ?it/s]/tmp/ipykernel_55814/1958088092.py:16: RuntimeWarning: invalid value encountered in sqrt
  return 1/(np.sqrt(2*np.pi*(var+eps))) * np.exp(-mahalD/2)
/tmp/ipykernel_55814/1958088092.py:16: RuntimeWarning: overflow encountered in exp
  return 1/(np.sqrt(2*np.pi*(var+eps))) * np.exp(-mahalD/2)
/tmp/ipykernel_55814/1958088092.py:60: RuntimeWarning: invalid value encountered in sqrt
  pi_by_var = -1*params['pi']/np.sqrt(params['var'] + eps)
100%|███████████████████████████████████████████████████████████████████████████████████████████████████▉| 999/1000 [1:06:53<00:04,  4.02s/it]

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

frame=  999 fps=164 q=-1.0 Lsize=    2070kB time=00:00:33.20 bitrate= 510.7kbits/s speed=5.46x    
video:2057kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.609714%
[libx264 @ 0x564a3f076800] frame I:4     Avg QP:18.37  size: 18484
[libx264 @ 0x564a3f076800] frame P:251   Avg QP:21.21  size:  4086
[libx264 @ 0x564a3f076800] frame B:744   Avg QP:25.68  size:  1352
[libx264 @ 0x564a3f076800] consecutive B-frames:  0.5%  0.4%  0.6% 98.5%
[libx264 @ 0x564a3f076800] mb I  I16..4: 15.5% 44.6% 39.9%
[libx264 @ 0x564a3f076800] mb P  I16..4:  0.9%  0.9%  1.4%  P16..4: 47.1% 16.9% 21.3%  0.0%  0.0%    skip:11.3%
[libx264 @ 0x564a3f076800] mb B  I16..4:  0.0%  0.0%  0.7%  B16..8: 27.8%  7.1%  3.9%  direct: 6.9%  skip:53.5%  L0:46.9% L1:46.9% BI: 6.2%
[libx264 @ 0x564a3f076800] 8x8 transform intra:24.2% inter:41.7%
[libx264 @ 0x564a3f076800] coded y,uvDC,uvAC intra: 67.6% 69.4% 55.3% inter: 15.8% 31.8% 8.8%
[libx264 @ 0x564a3f076800] i16 v,h,dc,p: 87%  6%  5%  1%


In [9]:
K = 5
T = 0.3
alpha = 0.1
os.mkdir(f'test_{K}_{T}_{alpha}')
main(f'test_{K}_{T}_{alpha}')

  0%|                                                                                                                | 0/1000 [00:00<?, ?it/s]/tmp/ipykernel_55814/1958088092.py:16: RuntimeWarning: invalid value encountered in sqrt
  return 1/(np.sqrt(2*np.pi*(var+eps))) * np.exp(-mahalD/2)
/tmp/ipykernel_55814/1958088092.py:16: RuntimeWarning: overflow encountered in exp
  return 1/(np.sqrt(2*np.pi*(var+eps))) * np.exp(-mahalD/2)
/tmp/ipykernel_55814/1958088092.py:60: RuntimeWarning: invalid value encountered in sqrt
  pi_by_var = -1*params['pi']/np.sqrt(params['var'] + eps)
100%|███████████████████████████████████████████████████████████████████████████████████████████████████▉| 999/1000 [1:03:49<00:03,  3.83s/it]


ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

frame=  999 fps=116 q=-1.0 Lsize=    7340kB time=00:00:33.20 bitrate=1811.1kbits/s speed=3.84x    
video:7328kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.163905%
[libx264 @ 0x55f53950d7c0] frame I:35    Avg QP:23.52  size: 13078
[libx264 @ 0x55f53950d7c0] frame P:321   Avg QP:27.32  size:  8317
[libx264 @ 0x55f53950d7c0] frame B:643   Avg QP:28.16  size:  6805
[libx264 @ 0x55f53950d7c0] consecutive B-frames: 12.4%  3.0%  6.9% 77.7%
[libx264 @ 0x55f53950d7c0] mb I  I16..4:  9.5% 24.0% 66.6%
[libx264 @ 0x55f53950d7c0] mb P  I16..4:  1.9% 11.6% 37.2%  P16..4: 11.8% 11.9%  8.1%  0.0%  0.0%    skip:17.5%
[libx264 @ 0x55f53950d7c0] mb B  I16..4:  1.4%  4.3% 22.3%  B16..8: 25.7% 15.7%  8.4%  direct: 5.2%  skip:17.0%  L0:49.1% L1:43.1% BI: 7.8%
[libx264 @ 0x55f53950d7c0] 8x8 transform intra:19.4% inter:17.4%
[libx264 @ 0x55f53950d7c0] coded y,uvDC,uvAC intra: 59.9% 55.9% 43.5% inter: 30.6% 13.1% 0.7%
[libx264 @ 0x55f53950d7c0] i16 v,h,dc,p: 43% 37% 17%  4%


In [10]:
K = 5
T = 0.3
alpha = 0.01
os.mkdir(f'test_{K}_{T}_{alpha}')
main(f'test_{K}_{T}_{alpha}')

  0%|                                                                                                                | 0/1000 [00:00<?, ?it/s]/tmp/ipykernel_55814/1958088092.py:16: RuntimeWarning: invalid value encountered in sqrt
  return 1/(np.sqrt(2*np.pi*(var+eps))) * np.exp(-mahalD/2)
/tmp/ipykernel_55814/1958088092.py:16: RuntimeWarning: overflow encountered in exp
  return 1/(np.sqrt(2*np.pi*(var+eps))) * np.exp(-mahalD/2)
/tmp/ipykernel_55814/1958088092.py:60: RuntimeWarning: invalid value encountered in sqrt
  pi_by_var = -1*params['pi']/np.sqrt(params['var'] + eps)
100%|███████████████████████████████████████████████████████████████████████████████████████████████████▉| 999/1000 [1:02:59<00:03,  3.78s/it]


ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

frame=  999 fps=142 q=-1.0 Lsize=    7432kB time=00:00:33.20 bitrate=1833.9kbits/s speed=4.71x    
video:7420kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.162815%
[libx264 @ 0x56099be45780] frame I:36    Avg QP:23.33  size: 13336
[libx264 @ 0x56099be45780] frame P:315   Avg QP:27.32  size:  8326
[libx264 @ 0x56099be45780] frame B:648   Avg QP:28.17  size:  6936
[libx264 @ 0x56099be45780] consecutive B-frames: 11.7%  3.6%  5.4% 79.3%
[libx264 @ 0x56099be45780] mb I  I16..4: 10.9% 22.0% 67.1%
[libx264 @ 0x56099be45780] mb P  I16..4:  1.8% 11.2% 36.8%  P16..4: 12.1% 12.2%  8.4%  0.0%  0.0%    skip:17.4%
[libx264 @ 0x56099be45780] mb B  I16..4:  1.4%  4.5% 22.8%  B16..8: 25.4% 15.6%  8.4%  direct: 5.1%  skip:17.0%  L0:48.9% L1:43.2% BI: 7.9%
[libx264 @ 0x56099be45780] 8x8 transform intra:19.1% inter:17.0%
[libx264 @ 0x56099be45780] coded y,uvDC,uvAC intra: 60.6% 56.8% 44.4% inter: 30.7% 13.3% 0.7%
[libx264 @ 0x56099be45780] i16 v,h,dc,p: 45% 35% 16%  4%


In [11]:
K = 4
T = 0.4
alpha = 0.01
os.mkdir(f'test_{K}_{T}_{alpha}')
main(f'test_{K}_{T}_{alpha}')

  0%|                                                                                                                | 0/1000 [00:00<?, ?it/s]/tmp/ipykernel_55814/1958088092.py:16: RuntimeWarning: invalid value encountered in sqrt
  return 1/(np.sqrt(2*np.pi*(var+eps))) * np.exp(-mahalD/2)
/tmp/ipykernel_55814/1958088092.py:16: RuntimeWarning: overflow encountered in exp
  return 1/(np.sqrt(2*np.pi*(var+eps))) * np.exp(-mahalD/2)
/tmp/ipykernel_55814/1958088092.py:60: RuntimeWarning: invalid value encountered in sqrt
  pi_by_var = -1*params['pi']/np.sqrt(params['var'] + eps)
100%|███████████████████████████████████████████████████████████████████████████████████████████████████▉| 999/1000 [1:03:26<00:03,  3.81s/it]


ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

frame=  999 fps=234 q=-1.0 Lsize=    3101kB time=00:00:33.20 bitrate= 765.2kbits/s speed=7.78x    
video:3089kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.389930%
[libx264 @ 0x562baab70ec0] frame I:14    Avg QP:21.46  size:  8782
[libx264 @ 0x562baab70ec0] frame P:330   Avg QP:25.91  size:  3929
[libx264 @ 0x562baab70ec0] frame B:655   Avg QP:27.15  size:  2661
[libx264 @ 0x562baab70ec0] consecutive B-frames: 10.4%  4.8%  5.1% 79.7%
[libx264 @ 0x562baab70ec0] mb I  I16..4: 17.6% 31.6% 50.8%
[libx264 @ 0x562baab70ec0] mb P  I16..4:  1.8% 14.0% 15.9%  P16..4: 11.7% 10.6%  5.3%  0.0%  0.0%    skip:40.9%
[libx264 @ 0x562baab70ec0] mb B  I16..4:  0.9%  2.8%  6.1%  B16..8: 27.6% 11.9%  4.2%  direct: 5.4%  skip:41.1%  L0:49.2% L1:43.4% BI: 7.3%
[libx264 @ 0x562baab70ec0] 8x8 transform intra:37.9% inter:15.9%
[libx264 @ 0x562baab70ec0] coded y,uvDC,uvAC intra: 42.2% 33.9% 25.0% inter: 15.8% 5.4% 0.5%
[libx264 @ 0x562baab70ec0] i16 v,h,dc,p: 51% 35% 13%  2%
[

In [12]:
K = 5
T = 0.2
alpha = 0.05
os.mkdir(f'test_{K}_{T}_{alpha}')
main(f'test_{K}_{T}_{alpha}')

  0%|                                                                                                                | 0/1000 [00:00<?, ?it/s]/tmp/ipykernel_55814/1958088092.py:16: RuntimeWarning: invalid value encountered in sqrt
  return 1/(np.sqrt(2*np.pi*(var+eps))) * np.exp(-mahalD/2)
/tmp/ipykernel_55814/1958088092.py:16: RuntimeWarning: overflow encountered in exp
  return 1/(np.sqrt(2*np.pi*(var+eps))) * np.exp(-mahalD/2)
/tmp/ipykernel_55814/1958088092.py:60: RuntimeWarning: invalid value encountered in sqrt
  pi_by_var = -1*params['pi']/np.sqrt(params['var'] + eps)
100%|███████████████████████████████████████████████████████████████████████████████████████████████████▉| 999/1000 [1:02:28<00:03,  3.75s/it]


ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

frame=  999 fps=107 q=-1.0 Lsize=    8460kB time=00:00:33.20 bitrate=2087.4kbits/s speed=3.56x    
video:8448kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.138057%
[libx264 @ 0x564076972f00] frame I:37    Avg QP:23.64  size: 13521
[libx264 @ 0x564076972f00] frame P:356   Avg QP:27.65  size:  9561
[libx264 @ 0x564076972f00] frame B:606   Avg QP:28.34  size:  7832
[libx264 @ 0x564076972f00] consecutive B-frames: 17.1%  4.2%  5.4% 73.3%
[libx264 @ 0x564076972f00] mb I  I16..4:  9.0% 23.7% 67.3%
[libx264 @ 0x564076972f00] mb P  I16..4:  1.7% 10.3% 41.1%  P16..4: 11.5% 11.9%  8.7%  0.0%  0.0%    skip:14.8%
[libx264 @ 0x564076972f00] mb B  I16..4:  1.4%  3.8% 25.9%  B16..8: 23.6% 15.6%  9.2%  direct: 5.1%  skip:15.4%  L0:49.0% L1:42.6% BI: 8.4%
[libx264 @ 0x564076972f00] 8x8 transform intra:16.6% inter:17.3%
[libx264 @ 0x564076972f00] coded y,uvDC,uvAC intra: 65.0% 62.5% 48.9% inter: 33.1% 15.1% 0.8%
[libx264 @ 0x564076972f00] i16 v,h,dc,p: 42% 35% 19%  5%
